In [1]:
import patsy as ps
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import patsy as ps

In [2]:
df_train = pd.read_csv('TRAIN.csv')
df_test = pd.read_csv('TEST.csv')

In [3]:
df_train.isnull().sum()

LID                0
MRP                0
CHANNEL            0
SELLER             0
ORATE              0
CRATE              0
OUPB               0
OLTERM             0
ODATE              0
FPD                0
LA                 0
TMTLM              0
RMTM               0
MDATE              0
LOTV               1
CLTV          163905
NB               198
DTI             6998
BCSAO           3981
CBCSAO         82221
FTHBI            218
LP                 0
PTYPE              0
NUNITS             0
OCCSTAT            0
STATE              0
MSA                0
ZIP                0
MIP           110261
CLDS               0
MIT           110261
RMI                0
FORECLOSED         0
NMONTHS            0
dtype: int64

since there are two many missing info for features: CLTV, MIP, MIT both in training and testing set, I decide to drop those three features.

### df_train = df_train.drop(['CLTV','MIP','MIT'],axis = 1)
df_test = df_test.drop(['CLTV','MIP','MIT'],axis = 1)
df_train

In [5]:
df_train.isnull().sum()

LID               0
MRP               0
CHANNEL           0
SELLER            0
ORATE             0
CRATE             0
OUPB              0
OLTERM            0
ODATE             0
FPD               0
LA                0
TMTLM             0
RMTM              0
MDATE             0
LOTV              1
NB              198
DTI            6998
BCSAO          3981
CBCSAO        82221
FTHBI           218
LP                0
PTYPE             0
NUNITS            0
OCCSTAT           0
STATE             0
MSA               0
ZIP               0
CLDS              0
RMI               0
FORECLOSED        0
NMONTHS           0
dtype: int64

Now, there are several feartures left with missing values: LOTV, NB, DTI, BCSAO, CBCSAO, FTHBI, I decide to deal with them individually, \
because the fit for them using same package might not be so accuracte for every features. And I found the testing set have similar \
situation with training set, thus, same operation should be processed with testing set also.

Feature: LOTV

In [6]:
df_train.groupby(['LOTV'])['LOTV'].count()

LOTV
4.0         4
5.0         5
6.0         5
7.0         8
8.0        13
        ...  
93.0     1034
94.0     1095
95.0    21377
96.0       61
97.0     5112
Name: LOTV, Length: 94, dtype: int64

Since for this column, there is only one missing value, and the data type is integer, so I don not need to do transformation for this column, \
I can instead fit the missing line using interpolate() function.

In [7]:
df_train['LOTV'].interpolate(method='linear', direction = 'forward', inplace=True)
df_test['LOTV'].interpolate(method='linear', direction = 'forward', inplace=True)
df_train['LOTV'].isnull().sum()

0

Feature: NB

In [8]:
df_train.groupby(['NB'])['NB'].count()

NB
1.0     65784
2.0     96484
3.0      1760
4.0       322
5.0        15
6.0         6
7.0         2
8.0         3
10.0        1
Name: NB, dtype: int64

For this column, the majority of data is 1.0 or 2.0, so I decide to use fillna method to fit the missing with median(mean is also feasible).

In [9]:
df_train['NB']= df_train['NB'].fillna(df_train['NB'].median())
df_test['NB']= df_test['NB'].fillna(df_test['NB'].median())
df_train['NB'].isnull().sum()

0

Feature: DTI BCSAO CBCSAO FTHBI

In [10]:
df_train.groupby(['DTI'])['DTI'].count()

DTI
1.0      31
2.0      47
3.0      96
4.0     134
5.0     139
       ... 
60.0    853
61.0    799
62.0    774
63.0    656
64.0    646
Name: DTI, Length: 64, dtype: int64

In [11]:
df_train.groupby(['BCSAO'])['BCSAO'].count()

BCSAO
350.0    2
354.0    2
372.0    1
375.0    1
400.0    7
        ..
834.0    1
842.0    1
844.0    1
845.0    1
850.0    1
Name: BCSAO, Length: 369, dtype: int64

In [12]:
df_train.groupby(['CBCSAO'])['CBCSAO'].count()

CBCSAO
396.0    1
400.0    4
446.0    1
455.0    1
459.0    1
        ..
824.0    4
826.0    2
828.0    1
832.0    1
834.0    1
Name: CBCSAO, Length: 340, dtype: int64

In [13]:
df_train.groupby(['FTHBI'])['FTHBI'].count()

FTHBI
N    137174
Y     27183
Name: FTHBI, dtype: int64

It looks the column FTHBI is not numerical values, we can make it to numerical by changing to 0 and 1 for this binary situation

In [14]:
df_train["FTHBI"] = df_train["FTHBI"].apply(lambda FTHBI: 0 if FTHBI == 'N' else (1 if FTHBI == 'Y' else None))
df_test["FTHBI"] = df_test["FTHBI"].apply(lambda FTHBI: 0 if FTHBI == 'N' else (1 if FTHBI == 'Y' else None))

By now, we have those four features with numerical values, and we can apply Multivariate Imputation by Chained Equations (MICE) \
to solve for missing values

In [15]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

df_imput = df_train[['DTI', 'BCSAO', 'CBCSAO', 'FTHBI']].copy()
imp = IterativeImputer()
imp.fit(df_imput)
imputed_df = imp.transform(df_imput)
imputed_df = pd.DataFrame(imputed_df, columns=df_imput.columns)

df_imput1 = df_test[['DTI', 'BCSAO', 'CBCSAO', 'FTHBI']].copy()
imp.fit(df_imput1)
imputed_df1 = imp.transform(df_imput1)
imputed_df1 = pd.DataFrame(imputed_df1, columns=df_imput.columns)

Now we have finished with missing values, we need go to deal with categorical features and timefeatures etc.

In [16]:
df_train

,LID,MRP,CHANNEL,SELLER,ORATE,CRATE,OUPB,OLTERM,ODATE,FPD,...,PTYPE,NUNITS,OCCSTAT,STATE,MSA,ZIP,CLDS,RMI,FORECLOSED,NMONTHS
0,8.118470e+11,3/1/2000,C,Suntrust Mortgage Inc.,8.375,8.375,58000,360,7/1/1999,9/1/1999,...,SF,1,P,OK,36420,730,0,N,False,12
1,9.288980e+11,2/1/2000,C,Amtrust Bank,7.500,7.500,204000,180,12/1/1999,2/1/2000,...,SF,1,P,WA,48300,988,0,N,False,21
2,2.229740e+11,1/1/2000,C,"Jpmorgan Chase Bank, Na",7.750,7.750,181000,360,12/1/1999,2/1/2000,...,SF,1,P,VA,40060,238,0,N,False,54
3,5.939500e+11,2/1/2000,B,"Bank Of America, N.A.",8.625,8.625,135000,360,1/1/2000,3/1/2000,...,PU,1,P,AZ,38060,852,0,N,False,16
4,6.773390e+11,2/1/2000,C,Other,7.750,7.750,189000,360,12/1/1999,2/1/2000,...,SF,1,P,IN,29140,479,0,N,False,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164570,8.194500e+11,2/1/2000,C,Amtrust Bank,7.875,7.875,106000,180,11/1/1999,1/1/2000,...,SF,1,I,MI,11460,481,0,N,False,24
164571,6.558220e+11,3/1/2000,R,"Jpmorgan Chase Bank, Na",8.000,8.000,144000,180,2/1/2000,4/1/2000,...,SF,1,P,CA,41740,920,0,N,False,19
164572,6.577330e+11,3/1/2000,R,First Tennessee Bank National Association,8.250,8.250,88000,360,2/1/2000,3/1/2000,...,SF,1,P,WV,26580,255,0,N,False,12
164573,3.145290e+11,1/1/2000,R,"Ge Mortgage Services, Llc",7.875,7.875,99000,240,10/1/1999,12/1/1999,...,SF,1,P,NY,35620,115,0,N,False,217


In [17]:
df_train = pd.concat([df_train.iloc[:,0:16],imputed_df,df_train.iloc[:,20:]], axis=1)
df_test = pd.concat([df_test.iloc[:,0:16],imputed_df1,df_test.iloc[:,20:]], axis=1)

In [18]:
df_train.isnull().sum()

LID           0
MRP           0
CHANNEL       0
SELLER        0
ORATE         0
CRATE         0
OUPB          0
OLTERM        0
ODATE         0
FPD           0
LA            0
TMTLM         0
RMTM          0
MDATE         0
LOTV          0
NB            0
DTI           0
BCSAO         0
CBCSAO        0
FTHBI         0
LP            0
PTYPE         0
NUNITS        0
OCCSTAT       0
STATE         0
MSA           0
ZIP           0
CLDS          0
RMI           0
FORECLOSED    0
NMONTHS       0
dtype: int64

there are some columns with binary values, lets start with the easiest first, RMI is the only one we need to fix

In [19]:
df_train["RMI"] = df_train["RMI"].apply(lambda RMI: 0 if RMI == 'N' else 1)
df_test["RMI"] = df_test["RMI"].apply(lambda RMI: 0 if RMI == 'N' else 1)
df_train.groupby(['RMI'])['RMI'].count()

RMI
0    162948
1      1627
Name: RMI, dtype: int64

Then I would like to deal with features with lots of categories : STATES SELLER ODATE FPD, I use the method group by counts

In [20]:
encodings = df_train.groupby(['SELLER'])['SELLER'].count().to_dict()
df_train['SELLER'] = df_train['SELLER'].map(encodings)
encodings = df_test.groupby(['SELLER'])['SELLER'].count().to_dict()
df_test['SELLER'] = df_test['SELLER'].map(encodings)

In [21]:
encodings = df_train.groupby(['STATE'])['STATE'].count().to_dict()
df_train['STATE'] = df_train['STATE'].map(encodings)
encodings = df_test.groupby(['STATE'])['STATE'].count().to_dict()
df_test['STATE'] = df_test['STATE'].map(encodings)

In [22]:
encodings = df_train.groupby(['ODATE'])['ODATE'].count().to_dict()
df_train['ODATE'] = df_train['ODATE'].map(encodings)
encodings = df_test.groupby(['ODATE'])['ODATE'].count().to_dict()
df_test['ODATE'] = df_test['ODATE'].map(encodings)

In [23]:
encodings = df_train.groupby(['FPD'])['FPD'].count().to_dict()
df_train['FPD'] = df_train['FPD'].map(encodings)
encodings = df_test.groupby(['FPD'])['FPD'].count().to_dict()
df_test['FPD'] = df_test['FPD'].map(encodings)

For the rest of the column features, the category is most to be 5, I decide to encode them together using label encoding method instead

In [24]:
other_categorical_cols = ['MRP','CHANNEL','LP','PTYPE','OCCSTAT',"CLDS"]
for col in df_train.columns:
    if col in other_categorical_cols:
        a={}
        for i in range(len(df_train[col].unique())):
            a[df_train[col].unique()[i]]=i
        work=df_train[col].map(a)
        df_train[col]=work

for col in df_test.columns:
    if col in other_categorical_cols:
        a={}
        for i in range(len(df_test[col].unique())):
            a[df_test[col].unique()[i]]=i
        work=df_test[col].map(a)
        df_test[col]=work

In [25]:
X_train = df_train.iloc[:,1:-2]
X_test = df_test.iloc[:,1:]
Y_train = df_train['NMONTHS']
Y_train = Y_train.to_numpy()

In [26]:
names_train = X_train.columns.values
names_test = X_test.columns.values

In [27]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_train = pd.DataFrame(X_train, columns = names_train )
X_test = ss.transform(X_test)
X_test = pd.DataFrame(X_test, columns = names_test )

In [28]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(random_state=0, tol=1e-5, n_jobs = -1)
Y_train_foreclosed = df_train['FORECLOSED']
LR.fit(X_train,Y_train_foreclosed)
train_acc = LR.score(X_train,Y_train_foreclosed)

In [29]:
test_pred = LR.predict(X_test)

In [30]:
FORCLOSED_predict = pd.DataFrame(test_pred,columns = ['FORCLOSED'])

In [31]:
FORCLOSED_predict.value_counts()

FORCLOSED
False        82282
True             6
dtype: int64

In [32]:
df_train['FORECLOSED'].value_counts()

False    162328
True       2247
Name: FORECLOSED, dtype: int64

In [33]:
from imblearn.over_sampling import SMOTE
from collections import Counter
oversample = SMOTE()
X, y = oversample.fit_resample(X_train,Y_train_foreclosed)
counter = Counter(y)
print(counter)

Counter({False: 162328, True: 162328})


In [34]:
LR.fit(X,y)
train_acc = LR.score(X,y)
train_acc 

0.7831704943078212

In [35]:
test_pred = LR.predict(X_test)

In [36]:
FORCLOSED_predict_balanced = pd.DataFrame(test_pred,columns = ['FORCLOSED'])
FORCLOSED_predict_balanced.value_counts()

FORCLOSED
False        57313
True         24975
dtype: int64

In [37]:
pred_proba = LR.predict_proba(X_test)[:,1]

In [38]:
pred_proba =  pd.DataFrame(pred_proba,columns = ['pre_proba'])

In [39]:
df_FORECLOSED = pd.concat([df_test.iloc[:,:1],FORCLOSED_predict_balanced,pred_proba], axis=1 )

In [40]:
#df_FORECLOSED.sort_values(by='pre_proba', ascending=False)

In [41]:
df_FORECLOSED_R = pd.concat([df_test.iloc[:,:1],pred_proba], axis=1 )
df_FORECLOSED_sorted = df_FORECLOSED_R.sort_values(by='pre_proba', ascending=False)
df_FORECLOSED_R

,LID,pre_proba
0,7.940000e+11,0.200389
1,2.770000e+11,0.375462
2,5.070000e+11,0.019029
3,4.930000e+11,0.282028
4,8.430000e+11,0.196717
...,...,...
82283,6.120000e+11,0.016836
82284,3.040000e+11,0.630622
82285,7.180000e+11,0.018502
82286,6.150000e+11,0.502205


In [42]:
df_FORECLOSED_sorted = df_FORECLOSED_sorted.reset_index(drop=True)
threshold = df_FORECLOSED_sorted['pre_proba'][999]
threshold

0.9484147788857582

In [43]:
df_FORECLOSED_R['FORECLOSED'] = df_FORECLOSED_R['pre_proba'].apply(lambda x: True if x >= threshold else False)

In [44]:
df_FORECLOSED_R['FORECLOSED'].value_counts()

False    81288
True      1000
Name: FORECLOSED, dtype: int64

In [45]:
df_FORECLOSED_R = df_FORECLOSED_R.drop(['pre_proba'],axis = 1)

In [46]:
df_FORECLOSED_R["FORECLOSED"].value_counts()

False    81288
True      1000
Name: FORECLOSED, dtype: int64

In [47]:
df_FORECLOSED_R.to_csv('FORECLOSERS.csv',index = False)

In [48]:
df_FORECLOSED.value_counts()

LID           FORCLOSED  pre_proba
1.000000e+11  False      0.001995     1
6.970000e+11  False      0.497665     1
6.980000e+11  False      0.157774     1
                         0.145775     1
                         0.140804     1
                                     ..
4.000000e+11  True       0.507387     1
              False      0.493117     1
                         0.477563     1
                         0.445534     1
1.000000e+12  True       0.959957     1
Length: 82288, dtype: int64

In [49]:
df_foreclosed = df_train.drop(['NMONTHS'],axis = 1)
df_foreclosed["FORECLOSED"] = df_foreclosed["FORECLOSED"].apply(lambda x: 0 if x == False else 1)

In [50]:
N=df_foreclosed.shape[0]
perm=np.random.permutation(range(N))
df_foreclosed=df_foreclosed.loc[perm]
Ntrain=int(.8*N)
Ntest=N-Ntrain
data_train=df_foreclosed.iloc[0:Ntrain]
data_test=df_foreclosed.iloc[Ntrain:N]
#data_train.astype('object').dtypes
#data_test.astype('object').dtypes


In [51]:
formula = "FORECLOSED~0+ORATE+NB+BCSAO+MDATE+OUPB+ZIP+CBCSAO+NUNITS+LOTV+CLDS+LP+STATE+ FPD+CRATE+OUPB"

#formula="FORECLOSED~0+np.log(ORATE)+np.log(NUNITS)+np.log(DTI)+np.log(BCSAO)+np.log(CBCSAO)+np.log(MDATE)+np.log(NB)+np.log(OUPB)\
#+MRP:CHANNEL+SELLER+ORATE*RMTM+OUPB*OLTERM+ODATE:NB+LP+TMTLM:ZIP+MDATE:LOTV+BCSAO:OUPB+CBCSAO:OUPB+LP*OUPB\
#+PTYPE+NUNITS:MSA+OCCSTAT+STATE*ZIP+NB:ORATE+ORATE:MDATE+OUPB:NB+MDATE:NUNITS+NUNITS**2+ZIP:LA+TMTLM**2"


#formula = "FORECLOSED~0+ORATE+CLDS+NUNITS+OCCSTAT+LP+CBCSAO+ZIP+FTHBI+CHANNEL+BCSAO+TMTLM+NB+FPD+OUPB+MSA+OLTERM+MRP+STATE+SELLER+MDATE+DTI+\
#np.log(ORATE)+np.log(NUNITS)+np.log(DTI)+np.log(BCSAO)+np.log(CBCSAO)"

#formula="FORECLOSED~0+np.log(ORATE)+ORATE+CLDS:LP+np.log(NUNITS)+np.log(FTHBI+LOTV)+OCCSTAT:ORATE+FTHBI*ORATE+LA*LP+PTYPE+np.log(LOTV)\
#+np.log(SELLER)+MRP:ORATE+CHANNEL*OLTERM+RMTM:LA+LOTV:LP+SELLER:STATE+STATE:LP"
Ytrain,Xtrain=ps.dmatrices(formula,data_train)
Ytest,Xtest=ps.dmatrices(formula,data_test)
Ntest=Ytest.shape[0]
reg1= LR.fit(Xtrain,Ytrain)
Ytest_pred=reg1.predict(Xtest)
MAD = np.mean(np.abs(Ytest-Ytest_pred))
print(MAD)
print(Xtrain)

/Users/bella/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.01482427278692725
[[8.1250e+00 1.0000e+00 7.1200e+02 ... 1.3330e+04 3.1341e+04 8.1250e+00]
 [8.5000e+00 2.0000e+00 7.2100e+02 ... 6.5460e+03 3.1341e+04 8.5000e+00]
 [8.6250e+00 1.0000e+00 7.7900e+02 ... 1.6776e+04 3.1341e+04 8.6250e+00]
 ...
 [8.0000e+00 2.0000e+00 7.7800e+02 ... 1.2800e+03 3.1341e+04 8.0000e+00]
 [7.6250e+00 1.0000e+00 7.6800e+02 ... 1.6776e+04 5.6182e+04 7.6250e+00]
 [7.8750e+00 2.0000e+00 7.5000e+02 ... 1.3330e+04 5.6182e+04 7.8750e+00]]


predict the result

In [52]:
X_test['FORECLOSED'] = df_FORECLOSED.FORCLOSED
Ytest,Xtest=ps.dmatrices(formula,X_test)

In [53]:
pred_proba_pasty = reg1.predict_proba(Xtest)[:,1]
pred_proba_pasty =  pd.DataFrame(pred_proba_pasty,columns = ['pre_proba_pasty'])
df_FORECLOSED_pasty = pd.concat([df_test.iloc[:,:1],pred_proba_pasty], axis=1 )
df_FORECLOSED_pasty_sorted = df_FORECLOSED_pasty.sort_values(by='pre_proba_pasty', ascending=False)
df_FORECLOSED_pasty 

,LID,pre_proba_pasty
0,7.940000e+11,0.494891
1,2.770000e+11,0.505737
2,5.070000e+11,0.489244
3,4.930000e+11,0.496226
4,8.430000e+11,0.502679
...,...,...
82283,6.120000e+11,0.471478
82284,3.040000e+11,0.504251
82285,7.180000e+11,0.503398
82286,6.150000e+11,0.501963


In [54]:
df_FORECLOSED_pasty_sorted = df_FORECLOSED_pasty_sorted.reset_index(drop=True)
threshold = df_FORECLOSED_pasty_sorted['pre_proba_pasty'][999]
threshold

0.5123861697817735

In [55]:
df_FORECLOSED_pasty['FORECLOSED'] = df_FORECLOSED_pasty['pre_proba_pasty'].apply(lambda x: True if x >= threshold else False)

In [56]:
df_FORECLOSED_pasty['FORECLOSED'].value_counts()

False    81288
True      1000
Name: FORECLOSED, dtype: int64

In [57]:
df_FORECLOSED_pasty = df_FORECLOSED_pasty.drop(["pre_proba_pasty"],axis = 1)

In [58]:
df_FORECLOSED_pasty

,LID,FORECLOSED
0,7.940000e+11,False
1,2.770000e+11,False
2,5.070000e+11,False
3,4.930000e+11,False
4,8.430000e+11,False
...,...,...
82283,6.120000e+11,False
82284,3.040000e+11,False
82285,7.180000e+11,False
82286,6.150000e+11,False


In [59]:
#df_FORECLOSED_pasty.to_csv('FORECLOSE.csv',index = False)